# 플랜
## 1. 전처리
1. 이상치 제거
2. 차원축소
    1. feature selection
        - 'repair_cost'
        - 'insure_cost'
        - 'repair_cost','insure_cost'
    2. PCA 할 feature 선택 \
       여러 feature를 선형조합하여 잠재변수(latent variable)를 만들 수 있다면 선택 가능
       - 사고관련 feature
4. 더미화
    - 샘플링과 더미화 split의 순서가?
    - Q)더미화를 하고 샘플링을 하면 카테고리 데이터가 오염될 수 있지 않은가?
3. train_test_split
    - 샘플링 전에 나눠져야 하는데?
3. sampling (oversampling)
    - SMOTE
    - ADASYM
    - RandomOverSampling

## 2. 학습
여러번 할거니 함수를 만들면서 해보자
1. 모델 고르기
2. 성능평가
3. 학습
3. CV
4. pipeline ?
    
## 3. try
1. 조건을 바꿔가며 시도
2. 함수 개선?

In [65]:
df = pd.read_csv('./insurance_fraud_detect_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fraud_YN               16000 non-null  int64  
 1   car_model              16000 non-null  int64  
 2   sharing_type           16000 non-null  int64  
 3   age_group              16000 non-null  int64  
 4   has_previous_accident  16000 non-null  int64  
 5   cumulative_use_count   16000 non-null  int64  
 6   b2b                    16000 non-null  int64  
 7   accident_ratio         16000 non-null  int64  
 8   pf_type                16000 non-null  int64  
 9   socarpass              16000 non-null  int64  
 10  socarsave              16000 non-null  int64  
 11  start_hour             16000 non-null  int64  
 12  duration               16000 non-null  int64  
 13  accident_hour          16000 non-null  int64  
 14  repair_cost            16000 non-null  float64
 15  in

# -- Base --

## 1. 전처리

### 1-1. 이상치 제거

In [ ]:
## None

### 1-2. 차원축소
#### feature selection

In [ ]:
## None

#### PCA

In [ ]:
## None

### 1-3. train test split

In [42]:
from sklearn.model_selection import train_test_split

In [66]:
def train_valid_test_split(df, test_size=0.2, random_state=13):
    train_set = df[df['test_set'] == 0]
    test_set = df[df['test_set'] == 1]

    train_set = train_set.drop('test_set', axis=1)
    test_set = test_set.drop('test_set', axis=1)

    X = train_set.drop('fraud_YN', axis=1)
    y = train_set['fraud_YN']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    X_eval, y_eval = test_set.drop('fraud_YN', axis=1), test_set['fraud_YN']
    
    print('y_train : ', np.unique(y_train, return_counts=True))
    print('y_test :', np.unique(y_test, return_counts=True))
    print('y_eval :', np.unique(y_eval, return_counts=True))
    
    return X_train, X_test, y_train, y_test, X_eval, y_eval

In [67]:
X_train, X_test, y_train, y_test, X_eval, y_eval = train_valid_test_split(df)

y_train :  (array([0, 1]), array([10276,    27]))
y_test : (array([0, 1]), array([2569,    7]))
y_eval : (array([0, 1]), array([3114,    7]))


### 1-3. sampling

In [ ]:
## None

### 1-4. dummy

In [70]:
def train_valid_test_dummy(X_train, X_test, X_eval, features=None):
    X_train_1hot = pd.get_dummies(X_train, columns=features)
    X_test_1hot = pd.get_dummies(X_test, columns=features)    
    X_eval_1hot = pd.get_dummies(X_eval, columns=features)  
    
    return X_train_1hot, X_test_1hot, X_eval_1hot

In [72]:
features = df.columns.drop(['fraud_YN', 'accident_ratio', 'repair_cost', 'insure_cost', 'repair_cnt', 'test_set'])
X_train_1hot, X_test_1hot, X_eval_1hot = train_valid_test_dummy(X_train, X_test, X_eval, features=features)
X_train_1hot.shape, X_test_1hot.shape, X_eval_1hot.shape

((10303, 75), (2576, 75), (3121, 75))

## 2. 학습

### 2-1. model
- Logistic Regression
- Decision Tree
- Random Forest
- LightGBM

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

In [78]:
models = [('LogisticReg', LogisticRegression(random_state=13, solver='liblinear')),
          ('DecisionTree', DecisionTreeClassifier(random_state=13, max_depth=4)),
          ('RandomForest', RandomForestClassifier(random_state=13, n_jobs=-1, n_estimators=100)),
          ('LightGBM', LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False))]

model_names = [model[0] for model in models]
model_names

['LogisticReg', 'DecisionTree', 'RandomForest', 'LightGBM']

### 2-2. 성능평가

In [40]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [82]:
def get_clf_eval(y_test, pred):
    acc = accuracy_score(y_test, pred)
    pre = precision_score(y_test, pred)
    rec = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    auc = roc_auc_score(y_test, pred)
    
    return acc, pre, rec, f1, auc

def print_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    acc, pre, rec, f1, auc = get_clf_eval(y_test, pred)
    
    print('=> confusion matrix')
    print(confusion)
    print('======================')
    
    print('Accurary: {0:.4f}, Precision: {1:.4f}'.format(acc, pre))
    print('Recall: {0:.4f}, F1: {1:.4f}, AUC:{2: .4f}'.format(rec, f1, auc))
    
def get_result(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    
    return get_clf_eval(y_valid, pred)

def get_result_pd(models, model_names, X_train, y_train, X_test, y_test):
    col_names = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    tmp = []
    
    for model in models:
        tmp.append(get_result(model[1], X_train, y_train, X_test, y_test))
        
    return pd.DataFrame(tmp, columns=col_names, index=model_names)

### 2-3. 학습

In [79]:
import time

In [83]:
st_time = time.time()

results = get_result_pd(models, model_names, X_train, y_train, X_test, y_test)

print('Fit time :', time.time() - st_time)
results

/Users/dockyum/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dockyum/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fit time : 1.3417472839355469


/Users/dockyum/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,accuracy,precision,recall,f1,roc_auc
LogisticReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomForest,0.997283,0.0,0.0,0.0,0.500000
LightGBM,0.997283,0.0,0.0,0.0,0.500000


# try-1 : sampling

In [86]:
df = pd.read_csv('./insurance_fraud_detect_data.csv')
df.shape

(16000, 25)

## 1. 전처리

### 1-1. 이상치 제거

In [ ]:
## None

### 1-2. 차원축소
#### feature selection

In [ ]:
## None

#### PCA

In [ ]:
## None

### 1-3. train test split

In [87]:
X_train, X_test, y_train, y_test, X_eval, y_eval = train_valid_test_split(df)

y_train :  (array([0, 1]), array([10276,    27]))
y_test : (array([0, 1]), array([2569,    7]))
y_eval : (array([0, 1]), array([3114,    7]))


### 1-3. sampling

In [88]:
from imblearn.over_sampling import RandomOverSampler

In [90]:
ROS = RandomOverSampler(random_state=13)
X_train_over, y_train_over = ROS.fit_resample(X_train, y_train)
np.unique(y_train_over, return_counts=True)

(array([0, 1]), array([10276, 10276]))

In [95]:
X_train.shape, y_train.shape

((10303, 23), (10303,))

In [96]:
X_train_over.shape, y_train_over.shape

((20552, 23), (20552,))

### 1-4. dummy

In [92]:
features = df.columns.drop(['fraud_YN', 'accident_ratio', 'repair_cost', 'insure_cost', 'repair_cnt', 'test_set'])
X_train_1hot, X_test_1hot, X_eval_1hot = train_valid_test_dummy(X_train_over, X_test, X_eval, features=features)
X_train_1hot.shape, X_test_1hot.shape, X_eval_1hot.shape

((20552, 75), (2576, 75), (3121, 75))

## 2. 학습

### 2-1. model
- Logistic Regression
- Decision Tree
- Random Forest
- LightGBM

### 2-2. 성능평가
- get_clf_eval
- print_clf_eval
- get_result
- get_result_pd

### 2-3. 학습

In [99]:
st_time = time.time()

results = get_result_pd(models, model_names, X_train_1hot, y_train_over, X_test_1hot, y_test)

print('Fit time :', time.time() - st_time)
results

/Users/dockyum/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fit time : 1.2923731803894043


/Users/dockyum/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,accuracy,precision,recall,f1,roc_auc
LogisticReg,0.073758,0.002925,1.000000,0.005833,0.535617
DecisionTree,0.520575,0.003236,0.571429,0.006436,0.545932
RandomForest,0.997283,0.000000,0.000000,0.000000,0.500000
LightGBM,0.997283,0.000000,0.000000,0.000000,0.500000
